In [1]:
using Plots

In [45]:
module NewtonRootSolver
using LinearAlgebra:norm
using IterativeSolvers:gmres

function forward_jacobian(F, x0, h, x)
    n = size(x0, 1)
    m = size(F(x0), 1)
    jac = zeros(n, m)

    function e(i)
        v = zeros(n)
        v[i] = 1
        return v
    end

    for i in 1:n
        for j in 1:m
            jac[i, j] = ((F(x + h*e(i)) - F(x)) / h)[j]
        end
    end

    return jac
end

function centered_jacobian(F, x0, h, x)
    n = size(x0, 1)
    m = size(F(x0), 1)
    jac = zeros(n, m)

    function e(i)
        v = zeros(n)
        v[i] = 1
        return v
    end

    for i in 1:n
        for j in 1:m
            jac[i, j] = ((F(x + h*e(i)) - F(x - h*e(i))) / 2h)[j]
        end
    end

    return jac
end

"""
    newtonsolve_backtrack(x0, F, JF, alpha0, kmax, Ftol, c1, rho, btmax)

Apply the Netwon method for nonlinear systems to find the zeros of function F

# Arguments

- `x0::AbstractVector{T}`: a column vector of n elements representing the
    starting point for the optimization method

- `F::Function`: a function that, for each column vector x, returns the vector
    of values F(x)

- `FDJ::Symbol`: `:fw`, `:c`, `:MF`. Forward, centered, and matrix-free
    approximations of the Jacobian matrix

- `h::AbstractFloat`: the finite difference step

- `alpha0::AbstractFloat`: a real number characterizing the step length of the
    optimization method (w.r.t. the merit function f) and the starting point of the
    backtracking strategy

- `kmax::Integer`: an integer scalar value characterizing the maximum number of
    iterations of the method

- `Ftol::AbstractFloat`: a real number characterizing the tolerance with
    respect to the norm of F in order to stop the method

- `c1::AbstractFloat`: the factor c1 for the Armijo condition that must be a
    scalar in (0, 1)

- `rho::AbstractFloat`: fixed factor, less that 1, used for reducing alpha0

- `btmax::Integer`: maximum number of steps for updating alpha during the
    backtracking strategy
"""
function newtonsolve_backtrack(
        x0::AbstractVector,
        F::Function,
        FDJ::Symbol,
        h::AbstractFloat,
        alpha0::AbstractFloat,
        kmax::Integer,
        Ftol::AbstractFloat,
        c1::AbstractFloat,
        rho::AbstractFloat,
        btmax::Integer
)
    JF = nothing
    JVP = nothing
    if FDJ == :fw
        JF = x -> forward_jacobian(F, x0, h, x)
    elseif FDJ == :c
        JF = x -> centered_jacobian(F, x0, h, x)
    elseif FDJ == :MF
        JVP = (x, p) -> (F(x + h*p) - F(x)) / h
    else
        error("Unknown FDJ: `$(FDJ)`")
    end
    
    f(x) = 0.5 * norm(F(x))^2 # merit function
    grad_f(x) = JF(x)'*F(x) # gradient of merit function
    
    xseq = []
    btseq = []

    k = 0
    xk = x0
    
    normFk = norm(F(xk))
    push!(xseq, xk)
    
    while true
        if normFk < Ftol
            break
        end
        
        if k > kmax
            break
        end
        
        delta_xk = nothing
        if FDJ == :MF
            # TODO(Andrea): build a linear operator using some package and pass
            #     that to the iterative solver (gmres)
            delta_xk = gmres(JVP, -F(xk))
        else
            delta_xk = JF(xk) \ (-F(xk))
        end
        
        k = k + 1
        
        # De Armijo backtracking
        alpha = alpha0
        bt = 1
        while true
            # TODO(Andrea): if using MF we need to change the armijo conditions
            if f(xk + alpha * delta_xk) <= (f(xk) + c1 * alpha * grad_f(xk)' * delta_xk)
                break
            end
            if bt > btmax
                break
            end
            
            alpha = rho * alpha
            bt = bt + 1
        end
        push!(btseq, bt)
        
        xk = xk + alpha * delta_xk
        
        normFk = norm(F(xk))
        push!(xseq, xk)
    end
    
    
    return xk, normFk, k, xseq, btseq
end

end
using Main.NewtonRootSolver

x0 = [-5.0, -5.0]
f1(x) = 2*x[1]-x[2]-exp(-x[1])
f2(x) = 2*x[2]-x[1]-exp(-x[2])
F(x) = [f1(x), f2(x)]
alpha0 = 1.
kmax = 5_000
Ftol = 1e-12
c1 = 0.0001
rho = 0.8
btmax = 50

println("Forward finite differences")
xk, normFk, k, xseq, btseq = NewtonRootSolver.newtonsolve_backtrack(x0, F, :fw, 1e-8, alpha0, kmax, Ftol, c1, rho, btmax)
@show xk
@show normFk
@show k
@show btseq
println()

println("Centered finite differences")
xk, normFk, k, xseq, btseq = NewtonRootSolver.newtonsolve_backtrack(x0, F, :c, 1e-8, alpha0, kmax, Ftol, c1, rho, btmax)
@show xk
@show normFk
@show k
@show btseq
println()

println("Matrix-free")
xk, normFk, k, xseq, btseq = NewtonRootSolver.newtonsolve_backtrack(x0, F, :MF, 1e-8, alpha0, kmax, Ftol, c1, rho, btmax)
@show xk
@show normFk
@show k
@show btseq
println()
nothing

Forward finite differences


xk = [0.5671432904097838, 0.5671432904097838]
normFk = 1.5700924586837752e-16
k = 9
btseq = Any[1, 1, 1, 1, 1, 1, 1, 1, 1]

Centered finite differences
xk = [0.5671432904097838, 0.5671432904097838]
normFk = 1.5700924586837752e-16
k = 9
btseq = Any[1, 1, 1, 1, 1, 1, 1, 1, 1]

Matrix-free


LoadError: MethodError: no method matching one(::Type{Any})
Closest candidates are:
  one(::Type{Union{Missing, T}}) where T at missing.jl:104
  one(!Matched::Type{Missing}) at missing.jl:103
  one(!Matched::BitArray{2}) at bitarray.jl:426
  ...